# NGC 3568 Image Reducer

The purpose of this notebook is to reduce the FLC files from Hubble by:

1. Aligning FLCs to the GAIA catalog
2. Drizzling Images together from a particular filter
3. Checking Drizzled images against the GAIA catalog

## Imports

In [ ]:
# Python Imports
import os
from os import path
from glob import glob, iglob
import tarfile

# 3rd Party Imports
from matplotlib import pyplot as plt

# Astropy Colab Imports
from astropy.io import fits
from astropy.wcs import WCS
from astropy.stats import SigmaClip
from astropy.table import QTable
from photutils.background import Background2D, SExtractorBackground as SEBack
from drizzlepac.tweakreg import TweakReg
from drizzlepac.astrodrizzle import AstroDrizzle
from astroscrappy import detect_cosmics as detectcrs

## Notebook Setup

### TweakReg Data Quality Flags

This cell defines the [DQ flags](https://www.stsci.edu/hst/instrumentation/acs/data-analysis/dq-flag-definitions) we want to ignore in the TweakReg process.

In [ ]:
# Data Directory
DATA_DIR = 'mastDownload/HST'

# FLC Glob Patter
FLC_GLOB_PAT    = path.join(DATA_DIR, '**/*flc.fits')
FLC_CR_GLOB_PAT = path.join(DATA_DIR, '**/*_cr_flc.fits')

# Backup Name
BACKUP_NAME = path.join(DATA_DIR, 'FLCs.tar.gz')

In [ ]:
# DQ Bits
DQ_BAD_DET  = 4
DQ_HOT_PIX  = 16
DQ_CR_PIX   = 4096+8192
DQ_GOOD_PIX = ~(DQ_BAD_DET + DQ_HOT_PIX + DQ_CR_PIX)

## Functions

In [ ]:
# Write Sources to Catalog
def writecat(outName: str, srcs: QTable, wcs: WCS):
    
    # Get Coords
    crds = wcs.pixel_to_world(srcs['xcentroid'], srcs['ycentroid'])
    
    # Open File for Writing
    with open(outName, 'w') as fid:
        
        for crd in crds:
            
            fid.write(f'{crd.icrs.ra.value:20.10f} {crd.icrs.dec.value:20.10f}\n')

### AstroDrizzle Backup

AstroDrizzle does not appear to be backing up the FLC files like it is supposed to. Therefore, we will write our own backup and restore.

In [ ]:
def preastrobackup():
    """This creates a compressed tarball of the FLC data before running AstroDrizzle."""
    
    # Open the tarball
    with tarfile.open(BACKUP_NAME, 'x:gz') as fid:
        
        # Add Discovered Files
        for filename in iglob(FLC_GLOB_PAT):
            fid.add(filename, arcname=path.basename(filename))

In [ ]:
def restoreastrobackup():
    """This resores the FLCs to their pre-drizzled state."""
    
    # Get the Existing File List
    fileList = glob(FLC_GLOB_PAT)
    
    # Open the TarFile and Extract Members
    with tarfile.open(BACKUP_NAME, 'r:gz') as fid:
        for filePath in fileList:
            fileName = path.basename(filePath)
            fid.extract(fileName)
            os.rename(fileName, filePath)

## Load the Data

In [ ]:
# Get the File Names and Sort them by filter
fileNameDict = {}
for fn in iglob(FLC_GLOB_PAT):
    
    # Open the file to get the filter
    with fits.open(fn) as hduList:
        hdr = hduList[0].header  # Get the Header
        if 'FILTER' in hdr:      # If the FILTER keyword exists (WFC3)
            filt = hdr['FILTER']
        elif 'CLEAR' not in hdr['FILTER1']:  # If FILTER1 is not clear (ACS)
            filt = hdr['FILTER1']
        else:                                # Else FILTER2 must be the filter (ACS)
            filt = hdr['FILTER2']
    
    # Store the Name using the filter as the dict key
    # Start the Empty List if Key does not exist
    if filt not in fileNameDict:
        fileNameDict[filt] = []
    fileNameDict[filt].append(fn)

## Align Images to GAIA

### Align F275W to GAIA

In [ ]:
# Image Find Parameters
imagefindcfg = refimagefindcfg = dict(
    # peakmax=900,
    threshold=100,
    conv_width=3.5,
    dqbits=DQ_GOOD_PIX
)

# Align the Images to the GAIA data
TweakReg(
    fileNameDict['F275W'],
    updatehdr=True,
    wcsname='GAIA',
    clean=True,
    configobj=None,
    refcat='../Data/GAIA/NGC3568-GAIA-RefCatalog-icrs.txt',
    runfile='F275W-Tweak.log',
    shiftfile=True,
    outshifts='F275W-Shifts.log',
    searchrad=0.1,
    minobj=15,
    tolerance=2.5,
    imagefindcfg=imagefindcfg
)

In [ ]:
# Clean Up
for fn in iglob('*.match'):
    os.remove(fn)
for fn in iglob('*.coo'):
    os.remove(fn)
for fn in iglob('*.list'):
    os.remove(fn)
os.rename('F275W-Shifts.log', 'tweak/F275W/F275W-Shifts.log')
os.rename('F275W-Tweak.log', 'tweak/F275W/F275W-Tweak.log')
os.rename('shifts_wcs.fits', 'tweak/F275W/shift_wcs.fits')

### Align F475W to GAIA

In [ ]:
# Image Find Parameters
imagefindcfg = refimagefindcfg = dict(
    # peakmax=900,
    threshold=40,
    conv_width=3.5,
    dqbits=DQ_GOOD_PIX
)

# Align the Images to the GAIA data
TweakReg(
    fileNameDict['F475W'],
    updatehdr=True,
    wcsname='GAIA',
    clean=True,
    configobj=None,
    refcat='../Data/GAIA/NGC3568-GAIA-RefCatalog-icrs.txt',
    runfile='F475W-Tweak.log',
    shiftfile=True,
    outshifts='F475W-Shifts.log',
    searchrad=0.1,
    minobj=15,
    tolerance=1,
    imagefindcfg=imagefindcfg
)

In [ ]:
# Clean Up
for fn in iglob('*.match'):
    os.remove(fn)
for fn in iglob('*.coo'):
    os.remove(fn)
for fn in iglob('*.list'):
    os.remove(fn)
os.rename('F475W-Shifts.log', 'tweak/F475W/F475W-Shifts.log')
os.rename('F475W-Tweak.log', 'tweak/F475W/F475W-Tweak.log')
os.rename('shifts_wcs.fits', 'tweak/F475W/shift_wcs.fits')

### Align F814W to GAIA

In [ ]:
# Image Find Parameters
imagefindcfg = refimagefindcfg = dict(
    # peakmax=900,
    threshold=40,
    conv_width=3.5,
    dqbits=DQ_GOOD_PIX
)

# Align the Images to the GAIA data
TweakReg(
    fileNameDict['F814W'],
    updatehdr=True,
    wcsname='GAIA',
    clean=True,
    configobj=None,
    refcat='../Data/GAIA/NGC3568-GAIA-RefCatalog-icrs.txt',
    runfile='F814W-Tweak.log',
    shiftfile=True,
    outshifts='F814W-Shifts.log',
    searchrad=0.1,
    minobj=15,
    tolerance=1,
    imagefindcfg=imagefindcfg
)

In [ ]:
# Clean Up
for fn in iglob('*.match'):
    os.remove(fn)
for fn in iglob('*.coo'):
    os.remove(fn)
for fn in iglob('*.list'):
    os.remove(fn)
os.rename('F814W-Shifts.log', 'tweak/F814W/F814W-Shifts.log')
os.rename('F814W-Tweak.log', 'tweak/F814W/F814W-Tweak.log')
os.rename('shifts_wcs.fits', 'tweak/F814W/shift_wcs.fits')

## Remove FLC CRs with AstroScrappy

* [Code Citation](https://zenodo.org/record/1482019)
* [Paper Citation](http://adsabs.harvard.edu/abs/2001PASP..113.1420V)

In [ ]:
# Backup First
preastrobackup()

In [ ]:
# Restore
restoreastrobackup()

Files are gzip'ed from the (bash) command line at the end of this section. If processing has already been done with the code below, it may be necessary to run

```bash
find -type f -name "*_cr_flc.fits.gz" -exec gunzip "{}" \;
```

in the top-level FLC folder (*i.e.*, `mastDownload/HST`).

### Remove F814W CRs

In [ ]:
# F814W Options
opts = {
    'sigclip'  : 4.5,
    'objlim'   : 10,
    'niter'    : 4,
    'cleantype': 'medmask',
    'sepmed'   : True,
    # 'fsmode'   : 'convolve',
    # 'psffwhm'  : 3.5,
    # 'psfsize'  : 9,
    'verbose'  : True
}

# Loop through FLCs to Process
for fn in fileNameDict['F814W']:
    
    # Out Name
    outName = fn.replace('flc.fits', 'cr_flc.fits')
    
    # Open Image
    with fits.open(fn) as hduList:
        
        # Get Cleaned Arrays
        crmsk1, crarr1 = detectcrs(hduList['SCI', 1].data, **opts)
        crmsk2, crarr2 = detectcrs(hduList['SCI', 2].data, **opts)
        
        # Write the Output Data
        outList = hduList.copy()  # Copy the Original File
        outList[0].header.add_history('CRs removed with AstroScrappy')  # Update comment about processing
        outList['SCI', 1].data = crarr1  # Change the old SCI image for CR Removed img
        outList.insert(4, fits.ImageHDU(crmsk1.astype(int), hduList['SCI', 1].header, 'MSK'))
        outList['SCI', 2].data = crarr2
        outList.insert(8, fits.ImageHDU(crmsk2.astype(int), hduList['SCI', 2].header, 'MSK'))
        outList.writeto(outName, overwrite=True)

### Remove F475W CRs

In [ ]:
# F814W Options
opts = {
    'sigclip'  : 4.5,
    'objlim'   : 10,
    'niter'    : 4,
    'cleantype': 'medmask',
    'sepmed'   : True,
    # 'fsmode'   : 'convolve',
    # 'psffwhm'  : 3.5,
    # 'psfsize'  : 9,
    'verbose'  : False
}

# Loop through FLCs to Process
for fn in fileNameDict['F475W']:
    
    # Out Name
    outName = fn.replace('flc.fits', 'cr_flc.fits')
    
    # Open Image
    with fits.open(fn) as hduList:
        
        # Get Cleaned Arrays
        crmsk1, crarr1 = detectcrs(hduList['SCI', 1].data, **opts)
        crmsk2, crarr2 = detectcrs(hduList['SCI', 2].data, **opts)
        
        # Write the Output Data
        outList = hduList.copy()  # Copy the Original File
        outList[0].header.add_history('CRs removed with AstroScrappy')  # Update comment about processing
        outList['SCI', 1].data = crarr1  # Change the old SCI image for CR Removed img
        outList.insert(4, fits.ImageHDU(crmsk1.astype(int), hduList['SCI', 1].header, 'MSK'))
        outList['SCI', 2].data = crarr2
        outList.insert(8, fits.ImageHDU(crmsk2.astype(int), hduList['SCI', 2].header, 'MSK'))
        outList.writeto(outName, overwrite=True)

### Remove F275W CRs

In [ ]:
# F814W Options
opts = {
    'sigclip'  : 4.0,
    'objlim'   : 7.5,
    'niter'    : 6,
    'cleantype': 'medmask',
    'sepmed'   : True,
    # 'fsmode'   : 'convolve',
    # 'psffwhm'  : 3.5,
    # 'psfsize'  : 9,
    'verbose'  : False
}

# Loop through FLCs to Process
for fn in fileNameDict['F275W']:
    
    # Out Name
    outName = fn.replace('flc.fits', 'cr_flc.fits')
    
    # Open Image
    with fits.open(fn) as hduList:
        
        # Get Cleaned Arrays
        crmsk1, crarr1 = detectcrs(hduList['SCI', 1].data, **opts)
        crmsk2, crarr2 = detectcrs(hduList['SCI', 2].data, **opts)
        
        # Write the Output Data
        outList = hduList.copy()  # Copy the Original File
        outList[0].header.add_history('CRs removed with AstroScrappy')  # Update comment about processing
        outList['SCI', 1].data = crarr1  # Change the old SCI image for CR Removed img
        outList.insert(4, fits.ImageHDU(crmsk1.astype(int), hduList['SCI', 1].header, 'MSK'))
        outList['SCI', 2].data = crarr2
        outList.insert(8, fits.ImageHDU(crmsk2.astype(int), hduList['SCI', 2].header, 'MSK'))
        outList.writeto(outName, overwrite=True)

## Drizzle Images for CR Correction

In [ ]:
# Update the File Name Dict
fileNameDict = {}
for fn in iglob(FLC_CR_GLOB_PAT):
    
    # Open the file to get the filter
    with fits.open(fn) as hduList:
        hdr = hduList[0].header  # Get the Header
        if 'FILTER' in hdr:      # If the FILTER keyword exists (WFC3)
            filt = hdr['FILTER']
        elif 'CLEAR' not in hdr['FILTER1']:  # If FILTER1 is not clear (ACS)
            filt = hdr['FILTER1']
        else:                                # Else FILTER2 must be the filter (ACS)
            filt = hdr['FILTER2']
    
    # Store the Name using the filter as the dict key
    # Start the Empty List if Key does not exist
    if filt not in fileNameDict:
        fileNameDict[filt] = []
    fileNameDict[filt].append(fn)

### AstroDrizzle ACS Data Quality Flags

[Link to ACS/WFC DQ Flags](https://www.stsci.edu/hst/instrumentation/acs/data-analysis/dq-flag-definitions)

In [ ]:
# DQ Bits
DQ_STAB_HOT = 16
DQ_WARM_PIX = 64
DQ_BAD_COL  = 128
DQ_FULL_WELL= 256
DQ_BADREF   = 512
DQ_SINK_PIX = 1024
DQ_GOOD_PIX = DQ_STAB_HOT + DQ_WARM_PIX + DQ_BAD_COL + DQ_FULL_WELL + DQ_SINK_PIX
# DQ_GOOD_PIX = ~4096

### Drizzle F814W Images

In [ ]:
# Drizzle Images
AstroDrizzle(
    fileNameDict['F814W'],
    output='F814W',
    runfile='F814W-Astro.log',
    wcskey='GAIA',
    context=False,
    configobj=None,
    num_cores=4,
    in_memory=False,
    build=True,
    restore=False,
    preserve=False,
    clean=True,
    skymethod='globalmin+match',
    driz_sep_scale=0.03,
    driz_sep_bits=DQ_GOOD_PIX,
    combine_type='iminmed',
    driz_cr_corr=False,
    final_wht_type='IVM',
    final_pixfrac=0.9,
    final_bits=DQ_GOOD_PIX,
    final_wcs=True,
    final_rot=0,
    final_scale=0.03,
    final_outnx=8500,
    final_outny=8500
)

### Drizzle F475W Images

In [ ]:
# Drizzle Images
AstroDrizzle(
    fileNameDict['F475W'],
    output='F475W',
    runfile='F475W-Astro.log',
    wcskey='GAIA',
    context=False,
    configobj=None,
    num_cores=4,
    in_memory=False,
    build=True,
    restore=False,
    preserve=False,
    clean=True,
    skymethod='globalmin+match',
    driz_sep_scale=0.03,
    driz_sep_bits=DQ_GOOD_PIX,
    combine_type='iminmed',
    driz_cr_corr=False,
    final_wht_type='IVM',
    final_pixfrac=0.9,
    final_bits=DQ_GOOD_PIX,
    final_wcs=True,
    final_refimage='F814W_drc.fits'
)

### AstroDrizzle WFC3 Data Quality Flags

[Link to WFC3/UVIS DQ Flags](https://hst-docs.stsci.edu/wfc3ihb/appendix-e-reduction-and-calibration-of-wfc3-data/e-1-the-stsci-reduction-and-calibration-pipeline#E.1TheSTScIReductionandCalibrationPipeline-tableE.2)

In [ ]:
# DQ Bits
DQ_STAB_HOT = 16
DQ_WARM_PIX = 64
DQ_BAD_COL  = 128
DQ_FULL_WELL= 256
DQ_BADREF   = 512
DQ_GOOD_PIX = DQ_STAB_HOT + DQ_WARM_PIX + DQ_FULL_WELL
# DQ_GOOD_PIX = ~4096

### Drizzle F275W Images

In [ ]:
# Drizzle Images
AstroDrizzle(
    fileNameDict['F275W'],
    output='F275W',
    runfile='F275W-Astro.log',
    wcskey='GAIA',
    context=False,
    configobj=None,
    num_cores=4,
    in_memory=False,
    build=True,
    restore=False,
    preserve=False,
    clean=True,
    skymethod='globalmin+match',
    driz_sep_scale=0.03,
    driz_sep_bits=DQ_GOOD_PIX,
    combine_type='iminmed',
    driz_cr_corr=False,
    final_wht_type='IVM',
    final_pixfrac=0.8,
    final_bits=DQ_GOOD_PIX,
    final_wcs=True,
    final_refimage='F814W_drc.fits'
)

## GZip CR Images

At this point, files are all gzip'ed in bash using

```bash
find -type f -name "*_cr_flc.fits" -exec gzip "{}" \;
```

in the top-level FLC folder (*i.e.*, `mastDownload/HST`).